# WPPNets training set color

# Tile

# LR 25*25 training set

In [1]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_tile_color/good/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate(liste_im_name):
    # define forward operator
    blur_width=2.0
    add_boundary=0
    kernel_size=16
    stride=4
    #my_layer=wgenpatex.gaussian_layer(kernel_size,blur_width,stride=stride)

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    #hr_img=tv_resize(600, antialias=True)((0.2989 * hr_img[:,0,:, :] + 0.5870 * hr_img[:,1, :, :] + 0.1140 * hr_img[:,2, :, :]).unsqueeze(1))

    # extraction des images 100*100
    #target_im2pat = wgenpatex.patch_extractor(100, pad=False,center=False,dim=2)
    target_im2pat = patch_extractor(100, pad=False,center=False)
    real_data = target_im2pat(hr_img, 6)
    
    # create (artificially) 25*25 LR observation and save it
    #def operator(inp):
        #if add_boundary==0:
            #return my_layer.forward(inp)
        #return my_layer.forward(inp[:,:,add_boundary:-add_boundary,add_boundary:-add_boundary])
    operator = Downsample(scale = 1/4, gaussian_std = 2)
    for j in range(6):
        lr_img=operator(torch.tensor(real_data[j,:].reshape(3,100,100),dtype=torch.float,device=DEVICE).view(1,3,100,100))
        lr_img+=0.01*torch.randn_like(lr_img)
        #lr_img=torch.tensor(real_data[j,:].reshape(3,100,100),dtype=torch.float,device=DEVICE).view(1,3,100,100)
        wgenpatex.imsave('training_img_WPPNets_tile_color/lr_tile/'+str(n)+'.png',lr_img) 
        n=n+1
    
    
    

/home/prof/smignon/anaconda3/envs/WPP_clone/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda
cpu
training_img_WPPNets_tile_color/good/000.png


/home/prof/smignon/anaconda3/envs/WPP_clone/lib/python3.7/site-packages/ipykernel_launcher.py:132: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this 

training_img_WPPNets_tile_color/good/001.png
training_img_WPPNets_tile_color/good/002.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

...

training_img_WPPNets_tile_color/good/225.png
training_img_WPPNets_tile_color/good/226.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

training_img_WPPNets_tile_color/good/227.png
training_img_WPPNets_tile_color/good/228.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

training_img_WPPNets_tile_color/good/229.png


# HR LR testing image

In [7]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_tile_color/test_hr_lr/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate(liste_im_name):
    # define forward operator
    #blur_width=2.0
    #add_boundary=0
    #kernel_size=16
    #stride=4
    operator = Downsample(scale = 1/4, gaussian_std = 2)
    #my_layer=wgenpatex.gaussian_layer(kernel_size,blur_width,stride=stride)

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    #hr_img=tv_resize(600, antialias=True)((0.2989 * hr_img[:,0,:, :] + 0.5870 * hr_img[:,1, :, :] + 0.1140 * hr_img[:,2, :, :]).unsqueeze(1))
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_tile_color/test_hr_lr/hr_tile.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,600,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('training_img_WPPNets_tile_color/test_hr_lr/lr_tile.png',lr_img)
    
    
    

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_tile_color/test_hr_lr/013.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


training_img_WPPNets_tile_color/test_hr_lr/hr_tile.png


# Reference HR

In [8]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_tile_color/ref_hr/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate([liste_im_name[1]]):

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_tile_color/ref_hr/ref_tile.png',hr_img)
    
    
    

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_tile_color/ref_hr/017.png


# Validation set 

In [10]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches

# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_tile_color/validation_set/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
# create (artificially) 25*25 LR observation and save it
operator = Downsample(scale = 1/4, gaussian_std = 2)

for i,name_im in enumerate(liste_im_name):
    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_tile_color/validation_tile/val_hr/validation_hr_tile_'+str(i+1)+'.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,100,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('training_img_WPPNets_tile_color/validation_tile/val_lr/validation_lr_tile_'+str(i+1)+'.png',lr_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_tile_color/validation_set/031.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


training_img_WPPNets_tile_color/validation_set/032.png


# Wood

# LR 25*25 training set

In [2]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
torch.cuda.set_device(2)
DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_wood_color/good/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate(liste_im_name):
    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    #hr_img=tv_resize(600, antialias=True)((0.2989 * hr_img[:,0,:, :] + 0.5870 * hr_img[:,1, :, :] + 0.1140 * hr_img[:,2, :, :]).unsqueeze(1))

    # extraction des images 100*100
    #target_im2pat = wgenpatex.patch_extractor(100, pad=False,center=False,dim=2)
    target_im2pat = patch_extractor(100, pad=False,center=False)
    real_data = target_im2pat(hr_img, 6)
    
    # create (artificially) 25*25 LR observation and save it
    #def operator(inp):
        #if add_boundary==0:
            #return my_layer.forward(inp)
        #return my_layer.forward(inp[:,:,add_boundary:-add_boundary,add_boundary:-add_boundary])
    operator = Downsample(scale = 1/4, gaussian_std = 2)
    for j in range(6):
        lr_img=operator(torch.tensor(real_data[j,:].reshape(3,100,100),dtype=torch.float,device=DEVICE).view(1,3,100,100))
        lr_img+=0.01*torch.randn_like(lr_img)
        #lr_img=torch.tensor(real_data[j,:].reshape(3,100,100),dtype=torch.float,device=DEVICE).view(1,3,100,100)
        wgenpatex.imsave('training_img_WPPNets_wood_color/lr_wood/'+str(n)+'.png',lr_img) 
        n=n+1
    
    
    

cpu
training_img_WPPNets_wood_color/good/000.png


/home/prof/smignon/anaconda3/envs/WPP_clone/lib/python3.7/site-packages/ipykernel_launcher.py:125: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this 

training_img_WPPNets_wood_color/good/001.png
training_img_WPPNets_wood_color/good/002.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

training_img_WPPNets_wood_color/good/003.png
training_img_WPPNets_wood_color/good/004.png


...

training_img_WPPNets_wood_color/good/242.png
training_img_WPPNets_wood_color/good/243.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

training_img_WPPNets_wood_color/good/244.png
training_img_WPPNets_wood_color/good/245.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0

training_img_WPPNets_wood_color/good/246.png


# HR LR testing image 

In [13]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_wood_color/test_hr_lr/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate(liste_im_name):
    # define forward operator
    #blur_width=2.0
    #add_boundary=0
    #kernel_size=16
    #stride=4
    operator = Downsample(scale = 1/4, gaussian_std = 2)
    #my_layer=wgenpatex.gaussian_layer(kernel_size,blur_width,stride=stride)

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    #hr_img=tv_resize(600, antialias=True)((0.2989 * hr_img[:,0,:, :] + 0.5870 * hr_img[:,1, :, :] + 0.1140 * hr_img[:,2, :, :]).unsqueeze(1))
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_wood_color/test_hr_lr/hr_wood.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,600,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('training_img_WPPNets_wood_color/test_hr_lr/lr_wood.png',lr_img)
    
    
    

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_wood_color/test_hr_lr/004.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Reference HR

In [14]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches
    
# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_wood_color/ref_hr/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
for i,name_im in enumerate([liste_im_name[1]]):

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_wood_color/ref_hr/ref_wood1.png',hr_img)
    
    
    

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_wood_color/ref_hr/016.png


# Validation set

In [15]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
#import wgenpatex_texture as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
#DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)

class patch_extractor(nn.Module):
    '''
    Module for creating custom patch extractor
    '''
    def __init__(self, patch_size, pad=False,center=False):
        super(patch_extractor, self).__init__()
        self.im2pat = nn.Unfold(kernel_size=patch_size)
        self.pad = pad
        self.padsize = patch_size-1
        self.center=center
        self.patch_size=patch_size

    def forward(self, input, batch_size=0):
        if self.pad:
            input = torch.cat((input, input[:,:,:self.padsize,:]), 2)
            input = torch.cat((input, input[:,:,:,:self.padsize]), 3)
        patches = self.im2pat(input).squeeze(0).transpose(1,0)
        if batch_size > 0:
            idx = torch.randperm(patches.size(0))[:batch_size]
            patches = patches[idx,:]
        if self.center:
            patches = patches - torch.mean(patches,-1).unsqueeze(-1)
        return patches

# chemin images 
liste_im_name   = [file for file in glob.glob("training_img_WPPNets_wood_color/validation_set/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
# create (artificially) 25*25 LR observation and save it
operator = Downsample(scale = 1/4, gaussian_std = 2)

for i,name_im in enumerate(liste_im_name):
    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('training_img_WPPNets_wood_color/validation_wood/val_hr/validation_hr_wood_'+str(i+1)+'.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,100,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('training_img_WPPNets_wood_color/validation_wood/val_lr/validation_lr_wood_'+str(i+1)+'.png',lr_img)

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
training_img_WPPNets_wood_color/validation_set/006.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


training_img_WPPNets_wood_color/validation_set/008.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


# Test set mean PSNR/LPIPS/SSIM

In [2]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)
    
# chemin images 
liste_im_name   = [file for file in glob.glob("test_img_WPPNets_color/wood/good/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
# create (artificially) 25*25 LR observation and save it
operator = Downsample(scale = 1/4, gaussian_std = 2)

for i,name_im in enumerate(liste_im_name):
    # define forward operator

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('test_img_WPPNets_color/wood/HR/hr'+str(i+1)+'.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,100,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('test_img_WPPNets_color/wood/LR/lr'+str(i+1)+'.png',lr_img) 
 

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
test_img_WPPNets_color/wood/good/000.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/wood/good/001.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


...

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/wood/good/017.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/wood/good/018.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


In [3]:
# This code belongs to the paper
#
# J. Hertrich, A. Houdard and C. Redenbach.
# Wasserstein Patch Prior for Image Superresolution.
# IEEE Transactions on Computational Imaging, 2022.
#
# Please cite the paper, if you use this code.
#
# This script applies the Wasserstein Patch Prior reconstruction onto the 2D SiC Diamonds image
# from Section 4.2 of the paper.
#
import argparse
import wgenpatex as wgenpatex
import torch
import skimage.transform
import numpy as np
from scipy.interpolate import griddata
import os
import lpips
import glob
from torchvision.transforms import Resize as tv_resize
from torch import nn
import math

#DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(DEVICE)
#torch.cuda.set_device(2)
DEVICE='cpu'
print(DEVICE)

class gaussian_downsample(nn.Module):
    '''
    Downsampling module with Gaussian filtering
    ''' 
    def __init__(self, kernel_size, sigma, stride, pad=False):
        super(gaussian_downsample, self).__init__()
        self.gauss = nn.Conv2d(3, 3, kernel_size, stride=stride, groups=3, bias=False) # in channels/ out channels     
        gaussian_weights = self.init_weights(kernel_size, sigma)
        self.gauss.weight.data = gaussian_weights.to(DEVICE)
        self.gauss.weight.requires_grad_(False)
        self.pad = pad
        self.padsize = kernel_size-1

    def forward(self, x):
        if self.pad:
            x = torch.cat((x, x[:,:,:self.padsize,:]), 2)
            x = torch.cat((x, x[:,:,:,:self.padsize]), 3)
        return self.gauss(x)

    def init_weights(self, kernel_size, sigma):
        x_cord = torch.arange(kernel_size)
        x_grid = x_cord.repeat(kernel_size).view(kernel_size, kernel_size)
        y_grid = x_grid.t()	
        xy_grid = torch.stack([x_grid, y_grid], dim=-1)
        mean = (kernel_size - 1)/2.
        variance = sigma**2.
        gaussian_kernel = (1./(2.*math.pi*variance))*torch.exp(-torch.sum((xy_grid - mean)**2., dim=-1)/(2*variance))
        gaussian_kernel = gaussian_kernel / torch.sum(gaussian_kernel)
        return gaussian_kernel.view(1, 1, kernel_size, kernel_size).repeat(3, 1, 1, 1)
    
def Downsample(scale = 0.25, gaussian_std = 2):
    ''' 
    downsamples an img by factor 4 using gaussian downsample from utils.py
    '''
    if scale > 1:
        print('Error. Scale factor is larger than 1.')
        return
    gaussian_std = gaussian_std
    kernel_size = 16
    gaussian_down = gaussian_downsample(kernel_size,gaussian_std,int(1/scale),pad=True) #gaussian downsample with zero padding
    return gaussian_down.to(DEVICE)
    
# chemin images 
liste_im_name   = [file for file in glob.glob("test_img_WPPNets_color/tile/good/*.png")]#.sort()
liste_im_name.sort()
n=0

# Extraction et resize
#for i,name_im in enumerate([liste_im_name[0]]):
# create (artificially) 25*25 LR observation and save it
operator = Downsample(scale = 1/4, gaussian_std = 2)

for i,name_im in enumerate(liste_im_name):
    # define forward operator

    
    # resize image 
    print(name_im)
    hr_img=wgenpatex.imread(name_im)
    hr_img=tv_resize(600, antialias=True)(hr_img)
    wgenpatex.imsave('test_img_WPPNets_color/tile/HR/hr'+str(i+1)+'.png',hr_img)
    
    lr_img=operator(hr_img.to('cpu'))#.view(1,1,100,100)
    lr_img+=0.01*torch.randn_like(lr_img)
    wgenpatex.imsave('test_img_WPPNets_color/tile/LR/lr'+str(i+1)+'.png',lr_img) 
 

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


cpu
test_img_WPPNets_color/tile/good/000.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/tile/good/001.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


...

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/tile/good/031.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


test_img_WPPNets_color/tile/good/032.png


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
